In [1]:
!pip install requests

In [2]:
import requests
import datetime

In [3]:
import base64

In [4]:
client_id = '592898ed445745d291732f0db3a7250c'
client_secret = 'b0d5223ecbce4328aea9f50dc2200c69'

In [5]:
class SpotifyAPI(object):
  access_token = None
  token_url = "https://accounts.spotify.com/api/token"
  access_token_expires = datetime.datetime.now()
  client_id = None
  access_token_did_expire = True
  client_secret = None
    
  def __init__(self,client_id, client_secret, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.client_id = client_id
    self.client_secret= client_secret

  def get_client_credentials(self):
    """
     Returns a base64 encoded string
    """
    client_id = self.client_id
    client_secret = self.client_secret
    if client_secret == None or client_id == None:
        raise Exception(" Set client_id and secret")
    client_creds = f"{client_id}:{client_secret}"
    client_creds_b64 = base64.b64encode(client_creds.encode())
    return client_creds_b64.decode()
  

  
  def get_token_headers(self):
    client_creds_b64 = self.get_client_credentials()
    return {
        'Authorization': f"Basic {client_creds_b64}"
    }

  def get_token_data(self):
        return{
             'grant_type': "client_credentials"
        }
    
  def perform_auth(self):
       token_url = self.token_url
       token_data = self.get_token_data()
       token_headers = self.get_token_headers()
       r = requests.post(token_url, data = token_data, headers=token_headers)
       if r.status_code not in range(200,299):
             return False
       data = r.json()
       now = datetime.datetime.now()
       access_token = data['access_token']
       self.access_token = access_token
       expires_in = data['expires_in'] #seconds
       expires = now+ datetime.timedelta(seconds=expires_in)
       self.access_token_expires = expires
       self.access_token_did_expire = expires < now
       return True


In [6]:
spotify = SpotifyAPI(client_id, client_secret)

In [7]:
spotify.perform_auth()


True

In [8]:
access_token = spotify.access_token


In [9]:
header = {
    "Authorization" : f"Bearer {access_token}"
}
endpoint = "https://api.spotify.com/v1/search"
data = {"q":"Time", "type": "track"}
r = requests.get(endpoint, data = data, headers = header)